** Unsupervised Classification of Land Covers in Sweden **

In this project, an unsupervised classification is done to separate various land covers. The study area is Sweden which is a diversity of land covers ranging from forests to agriculture.
Firstly, google earth engine module is installed, imported and authenticated the user.


In [1]:
#pip install earthengine-api --upgrade

In [2]:
import ee
import geemap


In [3]:
#ee.Authenticate()

In [4]:
try:
  ee.Initialize()
  print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
  print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


The image collection of Harmonized Sentinel-2 MSI with MultiSpectral Instrument in Level-2A of atmospheric correction is used. The date of captured images are in the summer of 2023.

In [5]:
# Load an image.
image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# overview the image 
print(image) 


ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "COPERNICUS/S2_SR_HARMONIZED"
      }
    }
  }
})


In [6]:
Map = geemap.Map()

Sentinel-2 satellite provides us with the 12 bands. To raise the separability, the false color composite is a common solution which uses bands of green, red and near infrared instead of RGB, respectively.

In [7]:
vizParams = {"bands": ['B3', 'B4', 'B8'],   "min": 0,   "max": 0.3}; 

Map.setCenter(18, 59.3, 8); 

Map.addLayer(image, vizParams, 'false color composite');
Map

Map(center=[59.3, 18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [8]:
# NAIP: National Agriculture Imagery Program
# Load four 2012 NAIP quarter quads, different locations.
naip2012 = ee.ImageCollection('USDA/NAIP/DOQQ')\
  .filterBounds(ee.Geometry.Rectangle(11.0273686052, 55.3617373725, 23.9033785336, 69.1062472602))\
  .filterDate('2023-06-21', '2023-09-21')



In [9]:
# Spatially mosaic the images in the collection and display.
mosaic = naip2012.mosaic()



In [10]:
Map = geemap.Map()
Map.setCenter(18, 59.3, 8)
Map.addLayer(mosaic, {}, 'spatial mosaic')
Map

Map(center=[59.3, 18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [11]:
# Stockholm 
center = ee.Geometry.Point([18, 59.3]).getInfo()['coordinates']

In [12]:
Map = geemap.Map()

# Center the map and display the image.
Map.setCenter(center[0], center[1], 10)
Map

Map(center=[59.3, 18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [13]:
#Gather images of Stockholm  

myCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(ee.Geometry.Point(center)) \
    .filterDate('2023-06-21', '2023-09-21') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)

In [14]:
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))

Number of images in the collection:  8


In [15]:
listOfImages

['20230727T100559_20230727T100559_T34VCL',
 '20230622T100601_20230622T100756_T33VXF',
 '20230710T101609_20230710T102211_T34VCL',
 '20230625T101601_20230625T101601_T33VXF',
 '20230625T101601_20230625T101601_T34VCL',
 '20230710T101609_20230710T102211_T33VXF',
 '20230905T100559_20230905T101047_T33VXF',
 '20230905T100559_20230905T101047_T34VCL']

In [16]:
img1 = myCollection.first()
img1.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [1830, 1830],
   'crs': 'EPSG:32634',
   'crs_transform': [60, 0, 300000, 0, -60, 6600000]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32634',
   'crs_transform': [10, 0, 300000, 0, -10, 6600000]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32634',
   'crs_transform': [10, 0, 300000, 0, -10, 6600000]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32634',
   'crs_transform': [10, 0, 300000, 0, -10, 6600000]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min':

In [17]:
vis_params = {"min": 0, 
              "max": 4000, 
              "bands": ["B3", "B4", "B8"]}  

Map.addLayer(img1, vis_params, "FirstImage", True) 
Map

Map(bottom=77435.0, center=[59.3, 18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

Oooops! It seems that the clouds are obstacle for the features to be seen. So, a correction will be done at the next paces.

In [18]:
# Get information about the bands as a list. 
bandNames = img1.bandNames() 
print('Band names: ', bandNames.getInfo()) # ee.List of band names 

Band names:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60']


In [19]:
# Get projection information from band 1. 
b1proj = img1.select('B1').projection() 
print('Band 1 projection: ', b1proj.getInfo()) # ee.Projection object 

Band 1 projection:  {'type': 'Projection', 'crs': 'EPSG:32634', 'transform': [60, 0, 300000, 0, -60, 6600000]}


In [20]:
# Get spatial resolution (in meters) information from bands
bscale = img1.select('B3').projection().nominalScale() 
print('Band 3 scale: ', bscale.getInfo()) # ee.Number 
bscale = img1.select('B4').projection().nominalScale() 
print('Band 4 scale: ', bscale.getInfo()) # ee.Number 
bscale = img1.select('B8').projection().nominalScale() 
print('Band 8 scale: ', bscale.getInfo()) # ee.Number 

Band 3 scale:  10
Band 4 scale:  10
Band 8 scale:  10


In [21]:
# Get a list of all metadata properties. 
properties = img1.propertyNames()
print('Metadata properties: ', properties.getInfo()) # ee.List of metadata properties

Metadata properties:  ['system:version', 'system:id', 'SPACECRAFT_NAME', 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE', 'BOA_ADD_OFFSET_B12', 'CLOUD_SHADOW_PERCENTAGE', 'system:footprint', 'SENSOR_QUALITY', 'GENERATION_TIME', 'CLOUDY_PIXEL_OVER_LAND_PERCENTAGE', 'CLOUD_COVERAGE_ASSESSMENT', 'THIN_CIRRUS_PERCENTAGE', 'GRANULE_MEAN_WV', 'BOA_ADD_OFFSET_B1', 'BOA_ADD_OFFSET_B2', 'DATASTRIP_ID', 'BOA_ADD_OFFSET_B5', 'BOA_ADD_OFFSET_B6', 'BOA_ADD_OFFSET_B3', 'BOA_ADD_OFFSET_B4', 'BOA_ADD_OFFSET_B9', 'BOA_ADD_OFFSET_B7', 'BOA_ADD_OFFSET_B8', 'GRANULE_ID', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', 'NOT_VEGETATED_PERCENTAGE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5', 'RADIOMETRIC_QUALITY', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1', 'HIGH_PROBA_CLOUDS_PERCENTAGE', 'UNCLASSIFIED_PERCENTAGE', 'OZONE_SO

In [22]:
# Get the timestamp and convert it to a date. 
ID = img1.get('DATATAKE_IDENTIFIER') 
print('Image ID: ', ID.getInfo()) # ee.Date

Image ID:  GS2B_20230727T100559_033367_N05.09


In [23]:
# Get a specific metadata property. 
cloudiness = img1.get('CLOUD_COVERAGE_ASSESSMENT')
print('CLOUD_COVERAGE_ASSESSMENT: ', cloudiness.getInfo()) # ee.Number 

CLOUD_COVERAGE_ASSESSMENT:  8.516913


In [24]:
img2 = myCollection.median()
Map.addLayer(img2, vis_params, "MedianImage", True) 
Map

Map(bottom=77435.0, center=[59.3, 18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

As one of the most common index, NDVI uses the radiation difference of vegetation in bands of red and near infrared and as a result it provides us with a separable map in which one can distinguish vegetation than the other land covers and even differentiate forest from grasslands. 

In [25]:
# Compute Normalized Difference Vegetation Index over S2-L2 product.
# NDVI = (NIR - RED) / (NIR + RED), where
# RED is B4, 664.5 nm
# NIR is B8, 835.1 nm

# Calculate the NDVI manually: NDVI = (B8 - B4) / (B8 + B4)
nir = img2.select('B8')
red = img2.select('B4')
ndvi = nir.subtract(red).divide(nir.add(red))

# Display the result. 
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']} 
Map.addLayer(ndvi, ndviParams, 'NDVI image #1')
Map

Map(bottom=77435.0, center=[59.3, 18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…

In [26]:
# NDVI using "normalizedDifference(NIR, RED)"
ndvi = img2.normalizedDifference(['B8', 'B4']); 

# Display the result. 
Map.addLayer(ndvi, ndviParams, 'NDVI image #2')
Map

Map(bottom=77435.0, center=[59.3, 18], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=…